In [ ]:
#default_exp encrypt

# encrypt
> need pycryptodome
Encrypt data with various methods eg AES

In [52]:
#hide
from nbdev.showdoc import *

In [53]:
#export
from Crypto.Cipher import AES
from beartype import beartype
from typing import Tuple, Optional
from hashlib import sha256

# Hashing

In [54]:
#export
def hashSha256(data:bytes, salt:Optional[bytes] = b'')->bytes:
  m=sha256()
  m.update(data)
  m.update(salt)
  return m.digest()

In [55]:
%%time
hashSha256(b'test')

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 14.5 µs


b'\x9f\x86\xd0\x81\x88L}e\x9a/\xea\xa0\xc5Z\xd0\x15\xa3\xbfO\x1b+\x0b\x82,\xd1]l\x15\xb0\xf0\n\x08'

# AES

## Encrypt

In [56]:
#export
@beartype
def encryptAes(key:bytes, data:bytes)-> Tuple[bytes,bytes]:
  '''
    note that key has to be 16 bits long
    response:
      cipherText in bytes
  '''
  cipher = AES.new(key, AES.MODE_EAX)
  ciphertext, tag = cipher.encrypt_and_digest(data)
  return ciphertext, cipher.nonce


In [57]:
key = b'testtesttesttest'
data = b'test'
password = b'testpw'
ciphertext, nonce = encryptAes(key,data)
print(ciphertext, nonce)

b'\xd9\x96\xce\xfd' b'wX7\xf7ZT\xcee\xab==\x99\x16\xf79\x9f'


### Encrypt with password

In [58]:
#export
@beartype
def encryptPasswordAes(password:bytes, data:bytes) ->Tuple[bytes,bytes]:
  key = hashSha256(password)[:16]
  return encryptAes(key=key,data=data)

In [59]:
pwData, pwNonce = encryptPasswordAes(password, data = b'this is a very important data')
print(pwData, pwNonce)

b"I'\x16'\xa3\x86\x95`\xf7k\x9f\xab\x92?z\xb1z=\x97\xa85\\U\x98\xd3@0\xbdr" b'jDg\xdbBm\x98\xd11[\xd16h`\xb4\xc2'


## Decrypt

In [60]:
#export
@beartype
def decryptAes(key:bytes, ciphertext:bytes, nonce:bytes)->bytes:
  cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
  plaintext = cipher.decrypt(ciphertext)
  return plaintext

In [61]:
decryptAes(key, ciphertext, nonce)

b'test'

### Decrypt password

In [62]:
@beartype
def decryptPasswordAes(ciphertext:bytes, password:bytes, nonce:bytes)->bytes:
  key = hashSha256(password)[:16]
  return decryptAes(key, ciphertext, nonce)
  

In [63]:
decryptPasswordAes(pwData, password, pwNonce)

b'this is a very important data'